Este notebook teve como objetivo ler a lista de palavras original e conjugar todos seus verbos regulares presentes
Fazendo um merge com a lista de palavras original, para recuperar a informação de "norm_gram", remover verbos irregulares, conjugar, e transformar as novas listas em um novo json utilizado para validação

## Dataframes de palavras adquiridas  
- words_df
    - dataframe com as atualmente em produção extraido de **words_file**

- lexico
    - dataframe completo de palavras, extraido de:   
    
- prod_in_lexico
    - filtro das palavras que hoje estão em produção no lexico completo (apos filtros realizados previamente, no arquivo **Words from Lexico_pt-br**)    
    
- verbs
    - lista de verbos em produção (filtrado atraves do merge prod_in_lexico)

- irregular_verbs 
    - listas de verbos irregulares    
   
- normal_verbs
    - lista de verbos retirando os irregulares
    
- final_conj
    - conjugando os verbos de normal_verbs 
    
- df_verbs_conj
    - df com apenas os verbos conjugados (sem as listas vazias quando a palavra não termina em ['ir', 'ar', 'er']
    
- list_of_verbs_conj
    - todas as listas de **df_verbs_conj** 
        
- df_final 
    - concatenação retirando duplicadas de final_conj, irregular verbs, prod_in_lexico
#### Objetivo: 
    - Criar um json como o words_df combinando as palavras na lista em final_conj, irregular verbs, prod_in_lexico

In [1]:
import json
import pandas as pd
import numpy as np

#### LENDO O JSON

In [2]:
#lendo o JSON 
f = open('Data/words.json') 
words_file = json.load(f)
f.close()

In [65]:
len(words_file)

154649

In [4]:
#transformando o sjson em um df
word_list = []
normalized_word_list =[]

for dic in words_file:
    word_list.append(dic['word'])
    normalized_word_list.append(dic['normalized'])
    
dic = {'word': word_list,
      'norm_word': normalized_word_list}

words_df = pd.DataFrame(dic, columns =['word', 'norm_word'])
words_df.head(10)

,word,norm_word
0,que,que
1,para,para
2,com,com
3,não,nao
4,uma,uma
5,por,por
6,dos,dos
7,como,como
8,mais,mais
9,foi,foi


------------

####  - Lendo dicionario completo

In [8]:
PATH = 'Data/dicionario/lexporbr_alfa_excel.xlsx'
lexico = pd.read_excel(PATH)
lexico = lexico.drop_duplicates(subset= 'ortografia', keep='first')
lexico.head()

,ortografia,cat_gram,inf_gram,freq_orto,freq_orto/M,log10_freq_orto,zipf_scale,zipf_rank,nb_letras,nb_homogr,...,bigramas,freq_bigramas,trigramas,freq_trigrama,inv_orto,inv_cvcv_orto,inv_bigra,inv_trigra,aleatorio,id
0,",",gram,pu,2132958,67976.940289,6.328982,7.841760,1,1,1,...,"#,_,#",72,"#,#",1,",",P,"#,_,#","#,#",0.186720,1
1,.,gram,pu,1603176,51092.895043,6.204981,7.717759,2,1,1,...,#._.#,164,#.#,1,.,P,#._.#,#.#,0.634065,2
2,de,gram,prp,1294610,41258.958999,6.112139,7.624916,3,2,5,...,#d_de_e#,66288,#de_de#,13795,ed,VC,#e_ed_d#,#ed_ed#,0.401176,3
3,o,gram,det,935286,29807.375755,5.970944,7.483722,4,1,5,...,#o_o#,45815,#o#,5,o,V,#o_o#,#o#,0.796133,4
4,a,gram,det,785941,25047.780794,5.895390,7.408167,5,1,11,...,#a_a#,64338,#a#,11,a,V,#a_a#,#a#,0.134573,5


In [9]:
lexico = lexico[['ortografia','cat_gram']]
lexico = lexico.rename(columns={'ortografia': 'word'})

In [10]:
lexico.head()

,word,cat_gram
0,",",gram
1,.,gram
2,de,gram
3,o,gram
4,a,gram


In [11]:
prod_in_lexico = pd.merge(left = words_df, right=lexico, how='left', on='word')
prod_in_lexico

,word,norm_word,cat_gram
0,que,que,gram
1,para,para,gram
2,com,com,gram
3,não,nao,adv
4,uma,uma,gram
...,...,...,...
154644,esmigalhado,esmigalhado,NaN
154645,deportamos,deportamos,NaN
154646,esgotarÃ¡,esgotara,NaN
154647,esgotarÃ£o,esgotarao,NaN


In [12]:
verbs = prod_in_lexico[prod_in_lexico.cat_gram == 'ver']
verbs.head()

,word,norm_word,cat_gram
9,foi,foi,ver
12,ser,ser,ver
19,tem,tem,ver
20,são,sao,ver
23,disse,disse,ver


#### tratando verbos irregulares

In [14]:
#html com lista de verbos irregulares de https://www.conjugacao.com.br/verbos-irregulares/
irregular_verbs_html = '''<div class="wrapper">
    <ul><li><a href="/verbo-abster/" title="Verbo abster">abster</a></li><li><a href="/verbo-acudir/" title="Verbo acudir">acudir</a></li><li><a href="/verbo-adequar/" title="Verbo adequar">adequar</a></li><li><a href="/verbo-aderir/" title="Verbo aderir">aderir</a></li><li><a href="/verbo-adjazer/" title="Verbo adjazer">adjazer</a></li><li><a href="/verbo-advertir/" title="Verbo advertir">advertir</a></li><li><a href="/verbo-advir/" title="Verbo advir">advir</a></li><li><a href="/verbo-afazer/" title="Verbo afazer">afazer</a></li><li><a href="/verbo-aferir/" title="Verbo aferir">aferir</a></li><li><a href="/verbo-agredir/" title="Verbo agredir">agredir</a></li><li><a href="/verbo-ansiar/" title="Verbo ansiar">ansiar</a></li><li><a href="/verbo-antedar/" title="Verbo antedar">antedar</a></li><li><a href="/verbo-antepor/" title="Verbo antepor">antepor</a></li><li><a href="/verbo-antever/" title="Verbo antever">antever</a></li><li><a href="/verbo-apor/" title="Verbo apor">apor</a></li><li><a href="/verbo-aprazer/" title="Verbo aprazer">aprazer</a></li><li><a href="/verbo-apropinquar/" title="Verbo apropinquar">apropinquar</a></li><li><a href="/verbo-aspergir/" title="Verbo aspergir">aspergir</a></li><li><a href="/verbo-assentir/" title="Verbo assentir">assentir</a></li><li><a href="/verbo-ater/" title="Verbo ater">ater</a></li><li><a href="/verbo-atrair/" title="Verbo atrair">atrair</a></li><li><a href="/verbo-atribuir/" title="Verbo atribuir">atribuir</a></li><li><a href="/verbo-auferir/" title="Verbo auferir">auferir</a></li><li><a href="/verbo-autodestruir/" title="Verbo autodestruir">autodestruir</a></li><li><a href="/verbo-avir/" title="Verbo avir">avir</a></li><li><a href="/verbo-bem-dizer/" title="Verbo bem-dizer">bem-dizer</a></li><li><a href="/verbo-bem-fazer/" title="Verbo bem-fazer">bem-fazer</a></li><li><a href="/verbo-bem-querer/" title="Verbo bem-querer">bem-querer</a></li><li><a href="/verbo-bendizer/" title="Verbo bendizer">bendizer</a></li><li><a href="/verbo-benfazer/" title="Verbo benfazer">benfazer</a></li><li><a href="/verbo-benquerer/" title="Verbo benquerer">benquerer</a></li><li><a href="/verbo-buir/" title="Verbo buir">buir</a></li><li><a href="/verbo-bulir/" title="Verbo bulir">bulir</a></li><li><a href="/verbo-caber/" title="Verbo caber">caber</a></li><li><a href="/verbo-cair/" title="Verbo cair">cair</a></li><li><a href="/verbo-cerzir/" title="Verbo cerzir">cerzir</a></li><li><a href="/verbo-circumpor/" title="Verbo circumpor">circumpor</a></li><li><a href="/verbo-circunver/" title="Verbo circunver">circunver</a></li><li><a href="/verbo-cobrir/" title="Verbo cobrir">cobrir</a></li><li><a href="/verbo-compelir/" title="Verbo compelir">compelir</a></li><li><a href="/verbo-competir/" title="Verbo competir">competir</a></li><li><a href="/verbo-compor/" title="Verbo compor">compor</a></li><li><a href="/verbo-comprazer/" title="Verbo comprazer">comprazer</a></li><li><a href="/verbo-concernir/" title="Verbo concernir">concernir</a></li><li><a href="/verbo-concluir/" title="Verbo concluir">concluir</a></li><li><a href="/verbo-condizer/" title="Verbo condizer">condizer</a></li><li><a href="/verbo-conferir/" title="Verbo conferir">conferir</a></li><li><a href="/verbo-confugir/" title="Verbo confugir">confugir</a></li><li><a href="/verbo-conseguir/" title="Verbo conseguir">conseguir</a></li><li><a href="/verbo-consentir/" title="Verbo consentir">consentir</a></li><li><a href="/verbo-construir/" title="Verbo construir">construir</a></li><li><a href="/verbo-consumir/" title="Verbo consumir">consumir</a></li><li><a href="/verbo-conter/" title="Verbo conter">conter</a></li><li><a href="/verbo-contradizer/" title="Verbo contradizer">contradizer</a></li><li><a href="/verbo-contrafazer/" title="Verbo contrafazer">contrafazer</a></li><li><a href="/verbo-contrair/" title="Verbo contrair">contrair</a></li><li><a href="/verbo-contrapor/" title="Verbo contrapor">contrapor</a></li><li><a href="/verbo-contrapropor/" title="Verbo contrapropor">contrapropor</a></li><li><a href="/verbo-contravir/" title="Verbo contravir">contravir</a></li><li><a href="/verbo-convergir/" title="Verbo convergir">convergir</a></li><li><a href="/verbo-convir/" title="Verbo convir">convir</a></li><li><a href="/verbo-crer/" title="Verbo crer">crer</a></li><li><a href="/verbo-cuspir/" title="Verbo cuspir">cuspir</a></li><li><a href="/verbo-dar/" title="Verbo dar">dar</a></li><li><a href="/verbo-decompor/" title="Verbo decompor">decompor</a></li><li><a href="/verbo-deferir/" title="Verbo deferir">deferir</a></li><li><a href="/verbo-delinquir/" title="Verbo delinquir">delinquir</a></li><li><a href="/verbo-denegrir/" title="Verbo denegrir">denegrir</a></li><li><a href="/verbo-depor/" title="Verbo depor">depor</a></li><li><a href="/verbo-desafazer/" title="Verbo desafazer">desafazer</a></li><li><a href="/verbo-desaguar/" title="Verbo desaguar">desaguar</a></li><li><a href="/verbo-desapor/" title="Verbo desapor">desapor</a></li><li><a href="/verbo-desaprazer/" title="Verbo desaprazer">desaprazer</a></li><li><a href="/verbo-desavir/" title="Verbo desavir">desavir</a></li><li><a href="/verbo-descaber/" title="Verbo descaber">descaber</a></li><li><a href="/verbo-descobrir/" title="Verbo descobrir">descobrir</a></li><li><a href="/verbo-descompor/" title="Verbo descompor">descompor</a></li><li><a href="/verbo-descomprazer/" title="Verbo descomprazer">descomprazer</a></li><li><a href="/verbo-desconstruir/" title="Verbo desconstruir">desconstruir</a></li><li><a href="/verbo-desconvir/" title="Verbo desconvir">desconvir</a></li><li><a href="/verbo-descrer/" title="Verbo descrer">descrer</a></li><li><a href="/verbo-desdar/" title="Verbo desdar">desdar</a></li><li><a href="/verbo-desdizer/" title="Verbo desdizer">desdizer</a></li><li><a href="/verbo-desimpedir/" title="Verbo desimpedir">desimpedir</a></li><li><a href="/verbo-desimpor/" title="Verbo desimpor">desimpor</a></li><li><a href="/verbo-deslinguar/" title="Verbo deslinguar">deslinguar</a></li><li><a href="/verbo-desmedir/" title="Verbo desmedir">desmedir</a></li><li><a href="/verbo-desmentir/" title="Verbo desmentir">desmentir</a></li><li><a href="/verbo-desmobiliar/" title="Verbo desmobiliar">desmobiliar</a></li><li><a href="/verbo-despedir/" title="Verbo despedir">despedir</a></li><li><a href="/verbo-despir/" title="Verbo despir">despir</a></li><li><a href="/verbo-despolir/" title="Verbo despolir">despolir</a></li><li><a href="/verbo-despor/" title="Verbo despor">despor</a></li><li><a href="/verbo-desprazer/" title="Verbo desprazer">desprazer</a></li><li><a href="/verbo-desprecaver/" title="Verbo desprecaver">desprecaver</a></li><li><a href="/verbo-desprover/" title="Verbo desprover">desprover</a></li><li><a href="/verbo-desquerer/" title="Verbo desquerer">desquerer</a></li><li><a href="/verbo-dessaber/" title="Verbo dessaber">dessaber</a></li><li><a href="/verbo-destruir/" title="Verbo destruir">destruir</a></li><li><a href="/verbo-desvaler/" title="Verbo desvaler">desvaler</a></li><li><a href="/verbo-desver/" title="Verbo desver">desver</a></li><li><a href="/verbo-deter/" title="Verbo deter">deter</a></li><li><a href="/verbo-devir/" title="Verbo devir">devir</a></li><li><a href="/verbo-diferir/" title="Verbo diferir">diferir</a></li><li><a href="/verbo-digerir/" title="Verbo digerir">digerir</a></li><li><a href="/verbo-disferir/" title="Verbo disferir">disferir</a></li><li><a href="/verbo-disperder/" title="Verbo disperder">disperder</a></li><li><a href="/verbo-dispor/" title="Verbo dispor">dispor</a></li><li><a href="/verbo-distrair/" title="Verbo distrair">distrair</a></li><li><a href="/verbo-divergir/" title="Verbo divergir">divergir</a></li><li><a href="/verbo-divertir/" title="Verbo divertir">divertir</a></li><li><a href="/verbo-dizer/" title="Verbo dizer">dizer</a></li><li><a href="/verbo-dormir/" title="Verbo dormir">dormir</a></li><li><a href="/verbo-embair/" title="Verbo embair">embair</a></li><li><a href="/verbo-emergir/" title="Verbo emergir">emergir</a></li><li><a href="/verbo-encobrir/" title="Verbo encobrir">encobrir</a></li><li><a href="/verbo-engolir/" title="Verbo engolir">engolir</a></li><li><a href="/verbo-entredizer/" title="Verbo entredizer">entredizer</a></li><li><a href="/verbo-entrefazer/" title="Verbo entrefazer">entrefazer</a></li><li><a href="/verbo-entreouvir/" title="Verbo entreouvir">entreouvir</a></li><li><a href="/verbo-entrepor/" title="Verbo entrepor">entrepor</a></li><li><a href="/verbo-entrequerer/" title="Verbo entrequerer">entrequerer</a></li><li><a href="/verbo-entrever/" title="Verbo entrever">entrever</a></li><li><a href="/verbo-entrevir/" title="Verbo entrevir">entrevir</a></li><li><a href="/verbo-entupir/" title="Verbo entupir">entupir</a></li><li><a href="/verbo-enxaguar/" title="Verbo enxaguar">enxaguar</a></li><li><a href="/verbo-enxerir/" title="Verbo enxerir">enxerir</a></li><li><a href="/verbo-equivaler/" title="Verbo equivaler">equivaler</a></li><li><a href="/verbo-escapulir/" title="Verbo escapulir">escapulir</a></li><li><a href="/verbo-esfazer/" title="Verbo esfazer">esfazer</a></li><li><a href="/verbo-estar/" title="Verbo estar">estar</a></li><li><a href="/verbo-estrear/" title="Verbo estrear">estrear</a></li><li><a href="/verbo-esvair/" title="Verbo esvair">esvair</a></li><li><a href="/verbo-expedir/" title="Verbo expedir">expedir</a></li><li><a href="/verbo-expelir/" title="Verbo expelir">expelir</a></li><li><a href="/verbo-expor/" title="Verbo expor">expor</a></li><li><a href="/verbo-extrapor/" title="Verbo extrapor">extrapor</a></li><li><a href="/verbo-fazer/" title="Verbo fazer">fazer</a></li><li><a href="/verbo-ferir/" title="Verbo ferir">ferir</a></li><li><a href="/verbo-flectir/" title="Verbo flectir">flectir</a></li><li><a href="/verbo-fletir/" title="Verbo fletir">fletir</a></li><li><a href="/verbo-fotocompor/" title="Verbo fotocompor">fotocompor</a></li><li><a href="/verbo-fraguar/" title="Verbo fraguar">fraguar</a></li><li><a href="/verbo-frigir/" title="Verbo frigir">frigir</a></li><li><a href="/verbo-fugir/" title="Verbo fugir">fugir</a></li><li><a href="/verbo-gelifazer/" title="Verbo gelifazer">gelifazer</a></li><li><a href="/verbo-gerir/" title="Verbo gerir">gerir</a></li><li><a href="/verbo-haver/" title="Verbo haver">haver</a></li><li><a href="/verbo-idear/" title="Verbo idear">idear</a></li><li><a href="/verbo-imergir/" title="Verbo imergir">imergir</a></li><li><a href="/verbo-impedir/" title="Verbo impedir">impedir</a></li><li><a href="/verbo-impelir/" title="Verbo impelir">impelir</a></li><li><a href="/verbo-impor/" title="Verbo impor">impor</a></li><li><a href="/verbo-incendiar/" title="Verbo incendiar">incendiar</a></li><li><a href="/verbo-indeferir/" title="Verbo indeferir">indeferir</a></li><li><a href="/verbo-indispor/" title="Verbo indispor">indispor</a></li><li><a href="/verbo-inferir/" title="Verbo inferir">inferir</a></li><li><a href="/verbo-influir/" title="Verbo influir">influir</a></li><li><a href="/verbo-ingerir/" title="Verbo ingerir">ingerir</a></li><li><a href="/verbo-insatisfazer/" title="Verbo insatisfazer">insatisfazer</a></li><li><a href="/verbo-inserir/" title="Verbo inserir">inserir</a></li><li><a href="/verbo-interdizer/" title="Verbo interdizer">interdizer</a></li><li><a href="/verbo-intermediar/" title="Verbo intermediar">intermediar</a></li><li><a href="/verbo-interpor/" title="Verbo interpor">interpor</a></li><li><a href="/verbo-interver/" title="Verbo interver">interver</a></li><li><a href="/verbo-intervir/" title="Verbo intervir">intervir</a></li><li><a href="/verbo-investir/" title="Verbo investir">investir</a></li><li><a href="/verbo-ir/" title="Verbo ir">ir</a></li><li><a href="/verbo-jazer/" title="Verbo jazer">jazer</a></li><li><a href="/verbo-justapor/" title="Verbo justapor">justapor</a></li><li><a href="/verbo-ler/" title="Verbo ler">ler</a></li><li><a href="/verbo-liquefazer/" title="Verbo liquefazer">liquefazer</a></li><li><a href="/verbo-maisquerer/" title="Verbo maisquerer">maisquerer</a></li><li><a href="/verbo-maldispor/" title="Verbo maldispor">maldispor</a></li><li><a href="/verbo-maldizer/" title="Verbo maldizer">maldizer</a></li><li><a href="/verbo-malfazer/" title="Verbo malfazer">malfazer</a></li><li><a href="/verbo-malinguar/" title="Verbo malinguar">malinguar</a></li><li><a href="/verbo-malparir/" title="Verbo malparir">malparir</a></li><li><a href="/verbo-malquerer/" title="Verbo malquerer">malquerer</a></li><li><a href="/verbo-manter/" title="Verbo manter">manter</a></li><li><a href="/verbo-mediar/" title="Verbo mediar">mediar</a></li><li><a href="/verbo-medir/" title="Verbo medir">medir</a></li><li><a href="/verbo-mentir/" title="Verbo mentir">mentir</a></li><li><a href="/verbo-minguar/" title="Verbo minguar">minguar</a></li><li><a href="/verbo-obter/" title="Verbo obter">obter</a></li><li><a href="/verbo-obvir/" title="Verbo obvir">obvir</a></li><li><a href="/verbo-odiar/" title="Verbo odiar">odiar</a></li><li><a href="/verbo-opor/" title="Verbo opor">opor</a></li><li><a href="/verbo-ouvir/" title="Verbo ouvir">ouvir</a></li><li><a href="/verbo-parir/" title="Verbo parir">parir</a></li><li><a href="/verbo-pedir/" title="Verbo pedir">pedir</a></li><li><a href="/verbo-perder/" title="Verbo perder">perder</a></li><li><a href="/verbo-perfazer/" title="Verbo perfazer">perfazer</a></li><li><a href="/verbo-perseguir/" title="Verbo perseguir">perseguir</a></li><li><a href="/verbo-persentir/" title="Verbo persentir">persentir</a></li><li><a href="/verbo-pleitear/" title="Verbo pleitear">pleitear</a></li><li><a href="/verbo-poder/" title="Verbo poder">poder</a></li><li><a href="/verbo-poer/" title="Verbo poer">poer</a></li><li><a href="/verbo-polir/" title="Verbo polir">polir</a></li><li><a href="/verbo-pospor/" title="Verbo pospor">pospor</a></li><li><a href="/verbo-por/" title="Verbo pôr">pôr</a></li><li><a href="/verbo-prazer/" title="Verbo prazer">prazer</a></li><li><a href="/verbo-predispor/" title="Verbo predispor">predispor</a></li><li><a href="/verbo-predizer/" title="Verbo predizer">predizer</a></li><li><a href="/verbo-preferir/" title="Verbo preferir">preferir</a></li><li><a href="/verbo-prepor/" title="Verbo prepor">prepor</a></li><li><a href="/verbo-pressentir/" title="Verbo pressentir">pressentir</a></li><li><a href="/verbo-pressupor/" title="Verbo pressupor">pressupor</a></li><li><a href="/verbo-preterir/" title="Verbo preterir">preterir</a></li><li><a href="/verbo-prevenir/" title="Verbo prevenir">prevenir</a></li><li><a href="/verbo-prever/" title="Verbo prever">prever</a></li><li><a href="/verbo-proferir/" title="Verbo proferir">proferir</a></li><li><a href="/verbo-progredir/" title="Verbo progredir">progredir</a></li><li><a href="/verbo-propor/" title="Verbo propor">propor</a></li><li><a href="/verbo-prosseguir/" title="Verbo prosseguir">prosseguir</a></li><li><a href="/verbo-prossupor/" title="Verbo prossupor">prossupor</a></li><li><a href="/verbo-prover/" title="Verbo prover">prover</a></li><li><a href="/verbo-provir/" title="Verbo provir">provir</a></li><li><a href="/verbo-pruir/" title="Verbo pruir">pruir</a></li><li><a href="/verbo-puir/" title="Verbo puir">puir</a></li><li><a href="/verbo-putrefazer/" title="Verbo putrefazer">putrefazer</a></li><li><a href="/verbo-querer/" title="Verbo querer">querer</a></li><li><a href="/verbo-raer/" title="Verbo raer">raer</a></li><li><a href="/verbo-rarefazer/" title="Verbo rarefazer">rarefazer</a></li><li><a href="/verbo-readequar/" title="Verbo readequar">readequar</a></li><li><a href="/verbo-reaver/" title="Verbo reaver">reaver</a></li><li><a href="/verbo-reavir/" title="Verbo reavir">reavir</a></li><li><a href="/verbo-recobrir/" title="Verbo recobrir">recobrir</a></li><li><a href="/verbo-recompor/" title="Verbo recompor">recompor</a></li><li><a href="/verbo-reconvir/" title="Verbo reconvir">reconvir</a></li><li><a href="/verbo-redar/" title="Verbo redar">redar</a></li><li><a href="/verbo-redispor/" title="Verbo redispor">redispor</a></li><li><a href="/verbo-redizer/" title="Verbo redizer">redizer</a></li><li><a href="/verbo-reexpedir/" title="Verbo reexpedir">reexpedir</a></li><li><a href="/verbo-reexpor/" title="Verbo reexpor">reexpor</a></li><li><a href="/verbo-refazer/" title="Verbo refazer">refazer</a></li><li><a href="/verbo-referir/" title="Verbo referir">referir</a></li><li><a href="/verbo-refletir/" title="Verbo refletir">refletir</a></li><li><a href="/verbo-refugir/" title="Verbo refugir">refugir</a></li><li><a href="/verbo-regredir/" title="Verbo regredir">regredir</a></li><li><a href="/verbo-reimpor/" title="Verbo reimpor">reimpor</a></li><li><a href="/verbo-reindispor/" title="Verbo reindispor">reindispor</a></li><li><a href="/verbo-reinserir/" title="Verbo reinserir">reinserir</a></li><li><a href="/verbo-reler/" title="Verbo reler">reler</a></li><li><a href="/verbo-remediar/" title="Verbo remediar">remediar</a></li><li><a href="/verbo-remedir/" title="Verbo remedir">remedir</a></li><li><a href="/verbo-reobter/" title="Verbo reobter">reobter</a></li><li><a href="/verbo-reouvir/" title="Verbo reouvir">reouvir</a></li><li><a href="/verbo-repedir/" title="Verbo repedir">repedir</a></li><li><a href="/verbo-repelir/" title="Verbo repelir">repelir</a></li><li><a href="/verbo-repetir/" title="Verbo repetir">repetir</a></li><li><a href="/verbo-repor/" title="Verbo repor">repor</a></li><li><a href="/verbo-repropor/" title="Verbo repropor">repropor</a></li><li><a href="/verbo-requerer/" title="Verbo requerer">requerer</a></li><li><a href="/verbo-resfolegar/" title="Verbo resfolegar">resfolegar</a></li><li><a href="/verbo-ressentir/" title="Verbo ressentir">ressentir</a></li><li><a href="/verbo-reter/" title="Verbo reter">reter</a></li><li><a href="/verbo-retrair/" title="Verbo retrair">retrair</a></li><li><a href="/verbo-retranspor/" title="Verbo retranspor">retranspor</a></li><li><a href="/verbo-rever/" title="Verbo rever">rever</a></li><li><a href="/verbo-revestir/" title="Verbo revestir">revestir</a></li><li><a href="/verbo-revir/" title="Verbo revir">revir</a></li><li><a href="/verbo-rir/" title="Verbo rir">rir</a></li><li><a href="/verbo-ruir/" title="Verbo ruir">ruir</a></li><li><a href="/verbo-saber/" title="Verbo saber">saber</a></li><li><a href="/verbo-sacudir/" title="Verbo sacudir">sacudir</a></li><li><a href="/verbo-sair/" title="Verbo sair">sair</a></li><li><a href="/verbo-santiguar/" title="Verbo santiguar">santiguar</a></li><li><a href="/verbo-satisfazer/" title="Verbo satisfazer">satisfazer</a></li><li><a href="/verbo-seguir/" title="Verbo seguir">seguir</a></li><li><a href="/verbo-sentir/" title="Verbo sentir">sentir</a></li><li><a href="/verbo-ser/" title="Verbo ser">ser</a></li><li><a href="/verbo-servir/" title="Verbo servir">servir</a></li><li><a href="/verbo-sobpor/" title="Verbo sobpor">sobpor</a></li><li><a href="/verbo-sobre-expor/" title="Verbo sobre-expor">sobre-expor</a></li><li><a href="/verbo-sobreexpor/" title="Verbo sobreexpor">sobreexpor</a></li><li><a href="/verbo-sobrepor/" title="Verbo sobrepor">sobrepor</a></li><li><a href="/verbo-sobrestar/" title="Verbo sobrestar">sobrestar</a></li><li><a href="/verbo-sobrevir/" title="Verbo sobrevir">sobrevir</a></li><li><a href="/verbo-sorrir/" title="Verbo sorrir">sorrir</a></li><li><a href="/verbo-sortear/" title="Verbo sortear">sortear</a></li><li><a href="/verbo-sortir/" title="Verbo sortir">sortir</a></li><li><a href="/verbo-sotopor/" title="Verbo sotopor">sotopor</a></li><li><a href="/verbo-subir/" title="Verbo subir">subir</a></li><li><a href="/verbo-submergir/" title="Verbo submergir">submergir</a></li><li><a href="/verbo-subpor/" title="Verbo subpor">subpor</a></li><li><a href="/verbo-subsumir/" title="Verbo subsumir">subsumir</a></li><li><a href="/verbo-subtrair/" title="Verbo subtrair">subtrair</a></li><li><a href="/verbo-sugerir/" title="Verbo sugerir">sugerir</a></li><li><a href="/verbo-sumir/" title="Verbo sumir">sumir</a></li><li><a href="/verbo-superexpor/" title="Verbo superexpor">superexpor</a></li><li><a href="/verbo-superimpor/" title="Verbo superimpor">superimpor</a></li><li><a href="/verbo-superpor/" title="Verbo superpor">superpor</a></li><li><a href="/verbo-supor/" title="Verbo supor">supor</a></li><li><a href="/verbo-suster/" title="Verbo suster">suster</a></li><li><a href="/verbo-telever/" title="Verbo telever">telever</a></li><li><a href="/verbo-ter/" title="Verbo ter">ter</a></li><li><a href="/verbo-torrefazer/" title="Verbo torrefazer">torrefazer</a></li><li><a href="/verbo-tossir/" title="Verbo tossir">tossir</a></li><li><a href="/verbo-trair/" title="Verbo trair">trair</a></li><li><a href="/verbo-transfazer/" title="Verbo transfazer">transfazer</a></li><li><a href="/verbo-transferir/" title="Verbo transferir">transferir</a></li><li><a href="/verbo-transfugir/" title="Verbo transfugir">transfugir</a></li><li><a href="/verbo-transgredir/" title="Verbo transgredir">transgredir</a></li><li><a href="/verbo-transpor/" title="Verbo transpor">transpor</a></li><li><a href="/verbo-traspor/" title="Verbo traspor">traspor</a></li><li><a href="/verbo-trazer/" title="Verbo trazer">trazer</a></li><li><a href="/verbo-treler/" title="Verbo treler">treler</a></li><li><a href="/verbo-tresler/" title="Verbo tresler">tresler</a></li><li><a href="/verbo-trespor/" title="Verbo trespor">trespor</a></li><li><a href="/verbo-tumefazer/" title="Verbo tumefazer">tumefazer</a></li><li><a href="/verbo-valer/" title="Verbo valer">valer</a></li><li><a href="/verbo-ver/" title="Verbo ver">ver</a></li><li><a href="/verbo-vestir/" title="Verbo vestir">vestir</a></li><li><a href="/verbo-vir/" title="Verbo vir">vir</a></li>    </ul>
</div>'''

#extraindo a lista
from bs4 import BeautifulSoup as soup
_div = soup(irregular_verbs_html, 'html.parser').find('div', {'class':'wrapper'})
irregular_verbs = [[i.text for i in b.find_all('li')] for b in _div.find_all('ul')][0]
irregular_verbs.append('quer')
irregular_verbs[0:10]

['abster',
 'acudir',
 'adequar',
 'aderir',
 'adjazer',
 'advertir',
 'advir',
 'afazer',
 'aferir',
 'agredir']

In [15]:
normal_verbs = verbs[~verbs.word.isin(irregular_verbs)]
normal_verbs

,word,norm_word,cat_gram
9,foi,foi,ver
19,tem,tem,ver
20,são,sao,ver
23,disse,disse,ver
32,pode,pode,ver
...,...,...,...
154575,há,ha,ver
154579,dá,da,ver
154585,vê,ve,ver
154587,ia,ia,ver


In [16]:
def verificar_ortografia(verbo):
    '''retorna se o verbo  termina em 'ir' 'er' 'ar' 
        None caso nao se encaixar nas regras
        '''    
    if len(verbo) < 4:
        return -1
    
    fim = verbo[-2:]
    if fim in ['ir', 'ar', 'er']:
        return fim
    
    return None

In [17]:
def conjuga_indicativos(row):      
    verbo = row['word']
    final = verificar_ortografia(verbo)   
    conjugacao = []
    rad = verbo[:-2]
   
    if final == 'ar':
        conjuga_ar = {
            'ar_pres' : ['o', 'as', 'a', 'amos', 'ais', 'am'] ,
            'ar_pret_perf' : ['ei', 'aste', 'ou', 'amos', 'aste', 'aram'],
            'ar_pret_imp' : ['ava', 'avas', 'ava', 'ávamos', 'ávais','avam'],
            'ar_mais_q_perf' : ['ara', 'aras', 'ara', 'áramos', 'áreis', 'aram'],
            'ar_fut_pres' : ['arei', 'arás', 'ará', 'aremos', 'areis', 'arão'],
            'ar_fut_pret' : ['aria', 'arias', 'aria', 'ariamos', 'arieis', 'ariam']
            }
        
        for key in conjuga_ar:
            for mode in conjuga_ar[key]:
                conjugacao.append(rad + mode)
                
        return {'conj': conjugacao}   
    
    elif final == 'er':
        conjuga_er = {
            'er_pres': ['o', 'es', 'e', 'emos', 'eis', 'em'],
            'er_pret_perf': ['i', 'este', 'eu', 'emos', 'estes', 'eram'],
            'er_pret_imp': ['ia', 'ias', 'ia', 'iamos', 'ieis', 'iam'],
            'er_mais_q_perf': ['era', 'eras', 'era', 'êramos', 'êreis', 'eram'],
            'er_fut_pres': ['erei', 'erás', 'erá', 'eremos', 'ereis', 'erão'],
            'er_fut_pret': ['eria', 'erias', 'eria', 'eriamos', 'erieis', 'eriam']
                }
        
        for key in conjuga_er:
            for mode in conjuga_er[key]:
                conjugacao.append(rad + mode)
        return {'conj':conjugacao}
    
    elif final == 'ir':
        conjuga_ir = {
            'ir_pres': ['o', 'es', 'e', 'imos', 'is', 'em'] ,
            'ir_pret_perf': ['i', 'iste', 'iu', 'imos', 'istes', 'iram'],
            'ir_pret_imp': ['ia', 'ias', 'ia', 'iamos', 'ieis', 'iam'],
            'ir_mais_q_perf': ['ira', 'iras', 'ira', 'íramos', 'ireis', 'iram'],
            'ir_fut_pres': ['irei', 'irás', 'irá', 'iremos', 'ireis', 'irão'],
            'ir_fut_pret': ['iria', 'irias', 'iria', 'iriamos', 'irieis', 'iriam']  
            }
        
        for key in conjuga_ir:
            for mode in conjuga_ir[key]:
                conjugacao.append(rad + mode)
                
        return {'conj': conjugacao}

    return {'conj': conjugacao}

In [18]:
conj = normal_verbs.apply(conjuga_indicativos, axis=1,result_type='expand')

In [19]:
final_conj = pd.concat([normal_verbs, conj], axis=1)
final_conj.sample(6)

,word,norm_word,cat_gram,conj
140515,pressiou,pressiou,ver,[]
51788,converteria,converteria,ver,[]
73525,enloqueceu,enloqueceu,ver,[]
76391,noosfera,noosfera,ver,[]
75118,infectaram,infectaram,ver,[]
93917,provarão,provarao,ver,[]


# criando df com as palavras conjugadas
  - transformando as listas de tempos verbais em linhas em um df

In [20]:
df_verbs_conj = final_conj[final_conj.conj.apply(lambda x: x != [])]
list_of_verbs_conj = df_verbs_conj.conj.to_list()

In [21]:
temp_list = []
for lista in list_of_verbs_conj:
    for verbo in lista:
        temp_dic = {}
        temp_dic['word'] = verbo
        temp_list.append(temp_dic)

In [22]:
df_de_verbos_conjugados = pd.DataFrame(temp_list)
# df_de_verbos_conjugados['norm_word'] = np.nan
df_de_verbos_conjugados.sample(5)

,word
85016,espetara
53012,embasara
149775,feminilizávamos
51011,encorajariam
6165,concorremos


In [23]:
# algumas conjugações tem a mesma escrita nas 1 e 3 pessoa: Eu partia, ele partia
print('conjugacoes repetidas:  ', df_de_verbos_conjugados.duplicated().sum())
df_de_verbos_conjugados[df_de_verbos_conjugados.duplicated()].head(5)

conjugacoes repetidas:   28736


,word
9,partimos
14,partia
20,partira
23,partiram
28,partireis


# Criando DF de irregular verbs

In [24]:
df_de_verbos_irregulares = pd.DataFrame(irregular_verbs, columns=['word'])
# df_de_verbos_irregulares['norm_word'] = np.nan
df_de_verbos_irregulares.sample(5)

,word
258,retranspor
66,delinquir
15,aprazer
170,ler
22,auferir


In [25]:
print('verbos irregulares repetidas: ', df_de_verbos_irregulares.duplicated().sum())
df_de_verbos_irregulares[df_de_verbos_irregulares.duplicated()]

verbos irregulares repetidas:  0


,word


# Palavras em produção

In [26]:
prod_in_lexico = prod_in_lexico[['word']]
prod_in_lexico.sample(5)

,word
136231,ontrário
53644,rinha
15328,reativação
96227,sorvendo
135866,odesastre


In [27]:
print('palavras repetidas atualmente em produção:: ', prod_in_lexico.duplicated().sum())
prod_in_lexico[prod_in_lexico.duplicated()]

palavras repetidas atualmente em produção::  8


,word
154592,de
154593,se
154594,no
154598,do
154602,um
154604,três
154637,novecentos
154642,hiper


# Misturando os 3 em apenas 1 retirando as repetições

In [28]:
df_final = pd.concat([prod_in_lexico, df_de_verbos_irregulares, df_de_verbos_conjugados], ignore_index=True)
df_final = df_final.drop_duplicates(subset='word', keep='first')
df_final = df_final.reset_index(drop=True)
df_final.sample(10)

,word
160920,checavas
20029,blitze
4651,opera
49932,lampreia
275428,workweará
158623,apontas
107853,cehgou
74255,filamentosa
82811,cercará
224399,assacava


In [29]:
df_final.shape

(275678, 1)

In [30]:
329601 - 275678 

53923

# REALIZANDO TRATATIVAS (NOVAMENTE) PARA TERMOS AS PALAVRAS NORMALIZADAS QUE BAGUNÇA AAAAA

In [31]:
import unicodedata
import re
import json

In [32]:
def strip_accents(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                    if unicodedata.category(c) != 'Mn')

In [35]:
norm_words = df_final.word.apply(lambda row: strip_accents(row))

In [52]:
df_norm_words = pd.DataFrame(norm_words.to_list(), columns=['norm_word'])
df_norm_words

,norm_word
0,que
1,para
2,com
3,nao
4,uma
...,...
275673,zonearia
275674,zonearias
275675,zoneariamos
275676,zonearieis


In [61]:
normalized = pd.concat([df_final, df_norm_words], axis= 1)
normalized

,word,norm_word
0,que,que
1,para,para
2,com,com
3,não,nao
4,uma,uma
...,...,...
275673,zonearia,zonearia
275674,zonearias,zonearias
275675,zoneariamos,zoneariamos
275676,zonearieis,zonearieis


# TRANSFORMANDO EM JSON PARA EXPORTAÇÃO

In [62]:
import json

i = 0
dic_aux = {}
list_aux = []

for word, norm_word in zip(normalized['word'], normalized['norm_word']):
    dic_aux = {
        'id': i,
        'word':word,
        'normalized': norm_word
    }    
    list_aux.append(dic_aux)
    i = i+1

In [66]:
#salvando o arquivo json
#with open('Data/words.json', 'w') as f:
#     json.dump(list_aux, f)

In [64]:
len(list_aux)

275678